# cache

Commands and population data for importing in to a database.

Even M$ visual studio products have to cache symbols somewhere.

I suppose this notebook should offline documentation too.


## notebook cache

Information about the current library cached.


### exports cache?

A cache of all exports, no matter the language.


#### the code

In [ ]:
// this is automatically replaced by the export cache script

#### create export cache?

## TODO: bookmark cache

Always wanted to write something like evernote to automatically pull in the content of pages I bookmark. Incognito mode on news sites to avoid payment. Ad block plugins installed.  Might as well make ad-blocked cached copies of every page I visit.




## TODO: documentation cache

Tools for searching and downloading documention as it related to this library.

